<a href="https://colab.research.google.com/github/pixelwizard2/Project.NLP--GuardianPay---Protecting-Against-Lost-Card-and-Copied-Card-Transactions/blob/main/GuardianPay_code_(%ED%95%9C%EA%B5%AD%EC%96%B4).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GuardianPay Project : 카드 결제 이상 행위 감지 및 보호 프로그램